**Introduction**

This Jupyter notebook provides a user-friendly interface for remotely controlling and monitoring [Pioreactors](https://pioreactor.com/en-ca), designed to facilitate yeast growth studies in various ethanol concentrations. Pioreactors enable controlled environmental conditions for microbial growth experiments, particularly useful for studying yeast's ethanol tolerance and the effects of different alcohol concentrations on yeast metabolism. Given the ability to control heating, stirring, liquid pumps, and gas valves and to monitor temperature, stirrer motor speed, and light scattering characteristics, the Pioreactor also has uses for chemistry reactions and other fields. By using a Jupyter notebook that's connected remotely to the [Pioreactor's web app hosted on Hugging Face Spaces](https://huggingface.co/spaces/AccelerationConsortium/PioReactor_gradio) you can execute commands, automate tasks, and view data from the PioReactor directly in this notebook without requiring physical interaction with the device. Although physical interaction will be need to change vials, add ingredients, etc.

This is based of [yeast- in-ethanol](https://docs.pioreactor.com/experiments/yeast-in-ethanol).

The API endpoints can we found at the bottom of the page through our Gradio interface. [Gradio-Interface](https://huggingface.co/spaces/AccelerationConsortium/PioReactor_gradio)

A more user friendly interface to interact with the Pioreactor can on Huggingface. [Streamlit-Interface](https://huggingface.co/spaces/AccelerationConsortium/PioReactor)

Install dependencies to allow for correct usage

In [1]:
%pip install gradio_client

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 318 kB 6.1 MB/s eta 0:00:01
     |████████████████████████████████| 179 kB 29.8 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 17.7 MB/s eta 0:00:01
     |████████████████████████████████| 121 kB 29.0 MB/s eta 0:00:01
     |████████████████████████████████| 447 kB 27.2 MB/s eta 0:00:01
     |████████████████████████████████| 70 kB 16.9 MB/s eta 0:00:01
     |████████████████████████████████| 90 kB 21.2 MB/s eta 0:00:01
     |████████████████████████████████| 167 kB 33.0 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 19.2 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 25.0 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 16.4 MB/s eta 0:00:01
     |████████████████████████████████| 172 kB 22.0 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 17.9 MB/s eta 0:00:01
     |███████████████████████████

**Apply imports and define client for connecting to web app**

In [ ]:
from gradio_client import Client

client = Client("AccelerationConsortium/PioReactor_gradio")

**API call to check the the current status of the PioReactor**

In [ ]:
result = client.predict(
		exp="Ed",
		api_name="/get_status_default"
)
print(result)

experiment = result[0] # Stores the current experiment from the results of the API call

**Create a new experiment for our example workflow and switch to it**

Creating a new experiment on a Pioreactor is like starting a "project" for a specific cultivation or research goal, with its settings and data isolated for that purpose. At the start nothing is active and no data is logged. These functions and process can be started through the API or directly through the Piocreactor interface if on the same network as the Pioreactor.

In [ ]:
new_experiment = "EXPERIMENT_NAME_HERE" # Change this to what you would like

result = client.predict(
		new_exp=new_experiment,
        exp=experiment,
		api_name="/new_experiment_default"
)

result = client.predict(
		ch_exp=new_experiment,
		exp=experiment,
		api_name="/change_experiment_default"
)

experiment = new_experiment

The vials should now be prepared. Follow the detailed procecures here [yeast- in-ethanol](https://docs.pioreactor.com/experiments/yeast-in-ethanol).

After you after done, execute the next block to start stirring and OD reading, you can optionally turn heating on by changing the boolean.

In [ ]:
heating = False
temp=30

rpm_speed = 500

result = client.predict(
		rpm=rpm_speed,
		experiment=experiment,
		state="start",
		api_name="/stirring_default"
)

result = client.predict(
		experiment=experiment,
		state="start",
		api_name="/od_default"
)

if heating:
    result = client.predict(
		temperature=30,
		experiment=experiment,
		state="start",
		option="Heat To Temp",
		api_name="/temp_automation_default"
)

After execution, you must wait for a desired amount of time, aprox 18 hours. After which you can execute the code block below to start the data logging.
You can change the variables of the experiment and start again.

In [ ]:
result = client.predict(
		rpm=0,
		experiment=experiment,
		state="stop",
		api_name="/stirring_default"
)

result = client.predict(
		experiment=experiment,
		state="stop",
		api_name="/od_default"
)

if heating:
    result = client.predict(
		temperature=30,
		experiment=experiment,
		state="stop",
		option="Heat To Temp",
		api_name="/temp_automation_default"
)

You can view the graphs by using the APIs below.

In [ ]:
import plotly.io as pio

# Time frame you want to see the data for (1 hour, 24 hour, All Data)
amount = "All Data"

result = client.predict(
		time_scale=amount,
		exp=experiment,
		api_name="/get_data_default"
)

# The results of the API call are go.Figure objects
# You can use the plotly library to display the data

temp = pio.from_json(result[0]["plot"])
temp.show()

od = pio.from_json(result[1]["plot"])
od.show()

norm_od = pio.from_json(result[2]["plot"])
norm_od.show()

growth_rate = pio.from_json(result[3]["plot"])
growth_rate.show()


**Notes**

All function calls in this notebook use api endpoints with `api_name="..._default"` this is so that we can utilize the example Pioreactor which we have setup. However, to create a custom workflow for you own Pioreactor replace use the api endpoints without the `_default` ending. Additional parameters will be needed, these should be the paramaters of your Pioreactor.